<a href="https://colab.research.google.com/github/jacquesescp/DSforAM_group10/blob/main/ReportGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install reportlab requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00


In [6]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, Image, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT, TA_JUSTIFY
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.colors import HexColor, black, red, blue, grey, darkslategray, whitesmoke, lightblue
from datetime import datetime
from google.colab import files
import os
from reportlab.lib.utils import ImageReader
from reportlab.lib.pagesizes import letter

LOGO_FILENAME = "sharkrisk_logo.png"
DASHBOARD_FILENAME = "RiskReporter.png"

SHARK_BLUE = HexColor('#003366')
ACCENT_BLUE = HexColor('#006699')
TEXT_COLOR = HexColor('#000000')
SHADED_COLOR = HexColor('#818589')

styles = getSampleStyleSheet()

def safe_add_style(name, **kwargs):
    if name in styles:
        styles[name].__dict__.update(kwargs)
    else:
        parent_style = kwargs.pop('parent', None)
        styles.add(ParagraphStyle(name=name, parent=parent_style, **kwargs))

# Style Definitions
safe_add_style('logo', fontName='Helvetica', fontSize=1, leading=1, alignment=TA_CENTER, textColor=ACCENT_BLUE, spaceAfter=0.001*inch)
safe_add_style('Tagline', fontName='Helvetica', fontSize=16, leading=20, alignment=TA_CENTER, textColor=ACCENT_BLUE, spaceAfter=0.2*inch, spaceBefore=0*inch)
safe_add_style('SectionHeading', parent=styles['h1'], fontName='Helvetica-Bold', fontSize=22, leading=26, textColor=SHARK_BLUE, spaceBefore=0.2*inch, spaceAfter=0.15*inch)
safe_add_style('SubSectionHeading', parent=styles['h2'], fontName='Helvetica-Bold', fontSize=16, leading=20, textColor=ACCENT_BLUE, spaceBefore=0.25*inch, spaceAfter=0.1*inch)
safe_add_style('BodyText', parent=styles['Normal'], fontName='Helvetica', fontSize=10, leading=14, textColor=TEXT_COLOR, alignment=TA_JUSTIFY, spaceAfter=0.1*inch, spaceBefore=0*inch)
safe_add_style('ExecutiveSummary', parent=styles['Normal'], fontName='Helvetica', fontSize=11, leading=16, textColor=TEXT_COLOR, alignment=TA_JUSTIFY, spaceAfter=0.05*inch)
safe_add_style('DateFooter', parent=styles['BodyText'], alignment=TA_CENTER)
safe_add_style('TableText', parent=styles['Normal'], fontName='Helvetica', fontSize=10, leading=14, textColor=TEXT_COLOR, alignment=TA_LEFT, spaceAfter=0.1*inch)
safe_add_style('BulletPoint', parent=styles['Normal'], leftIndent=0.25 * inch, bulletIndent=0.05 * inch, bulletText='\u2022', alignment=TA_LEFT, spaceBefore=3, spaceAfter=3, fontSize=10)

def _header_footer(canvas, doc):
    canvas.saveState()

    if doc.page > 1:
        page_width = letter[0]
        text_start_x = doc.leftMargin
        text_end_x = page_width - doc.rightMargin
        center_x = page_width / 2

        canvas.setFont('Helvetica', 9)
        canvas.setFillColor(SHADED_COLOR)

        footer_y = 0.35 * inch
        canvas.drawString(text_start_x, footer_y, "SharkRisk - Confidential Report")
        canvas.drawCentredString(center_x, footer_y, f"Page {doc.page}")
        header_y = letter[1] - 0.35 * inch
        canvas.drawRightString(text_end_x, header_y, "SharkRisk: Python Risk Management Library Presentation")
        line_y = header_y - 0.05 * inch
        canvas.setStrokeColor(ACCENT_BLUE)
        canvas.setLineWidth(0.5)
        canvas.line(text_start_x, line_y, text_end_x, line_y)

    canvas.restoreState()


# TABLES
def get_feature_impact_table():
    table_style = styles['TableText']
    data = [
        ['Feature', 'Impact'],

        [Paragraph("Comprehensive & Accurate Calculation", table_style),
         Paragraph("Instantly measures Value at Risk (VaR), Credit Exposure, and Greeks (Delta, Gamma, Vega, etc.), going beyond existing partial libraries (e.g., QF-Lib).", table_style)],

        [Paragraph("AI Sentiment Integration", table_style),
         Paragraph("Incorporates real-time AI/NLP sentiment analysis (Reddit, Twitter) to anticipate market shocks and sentiment-driven volatility spikes.", table_style)],

        [Paragraph("Proactive Hedging Strategies", table_style),
         Paragraph("Actively proposes targeted hedging methods (options, futures) to neutralize vulnerabilities identified by the risk metrics.", table_style)],

        [Paragraph("Automated Reporting", table_style),
         Paragraph("Instant generation of complete, transparent, and compliance-ready reports for investors and regulatory institutions.", table_style)]
    ]

    table_width = letter[0] - 2 * inch
    col_widths = [table_width * 0.30, table_width * 0.70] # 30% for Feature, 70% for Impact
    table = Table(data, colWidths=col_widths)
    table.setStyle(TableStyle([
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
        ('BOX', (0,0), (-1,-1), 1, colors.black),
        ('BACKGROUND', (0,0), (-1,0), ACCENT_BLUE), # Using ACCENT_BLUE for header
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
        ('ALIGN', (0,0), (-1,0), 'CENTER'),
        ('VALIGN', (0,0), (-1,0), 'MIDDLE'),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0,0), (-1,0), 8),
        ('VALIGN', (0,1), (-1,-1), 'TOP'), # Top align content
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('BACKGROUND', (0,1), (-1,-1), colors.white),
    ]))

    return [table, Spacer(1, 0.1*inch)]


def get_main_functions_table():
    table_text_style = styles['TableText']
    data = [
        ['#', 'Function', 'Purpose', 'Value Proposition'],

        ['1', Paragraph("Import Portfolio", table_text_style),
         Paragraph("Establishes a clean, validated, and structured dataset for risk analysis.", table_text_style),
         Paragraph("Ensures data quality, historical depth, and structural validity (weights, tickers) before any computation.", table_text_style)],

        ['2', Paragraph("Risk Metrics", table_text_style),
         Paragraph("Calculates a comprehensive suite of portfolio and derivative-level risk measures.", table_text_style),
         Paragraph("Delivers all critical metrics (VaR, Greeks, Credit Exposure) in one place, moving beyond basic volatility measures.", table_text_style)],

        ['3', Paragraph("Sentiment Analysis", table_text_style),
         Paragraph("Leverages AI to incorporate qualitative market factors into risk assessment.", table_text_style),
         Paragraph("Predicts short-term volatility and risk driven by social media sentiment, adding an alpha factor to risk monitoring.", table_text_style)],

        ['4', Paragraph("Hedging Strategies", table_text_style),
         Paragraph("Proposes tactical measures to mitigate identified risks.", table_text_style),
         Paragraph("Shifts the risk process from passive measurement to proactive, data-driven risk reduction strategies.", table_text_style)],

        ['5', Paragraph("Automated Reporting", table_text_style),
         Paragraph("Generates professional reports for investors and compliance bodies.", table_text_style),
         Paragraph("Streamlines communication and regulatory adherence by instantly translating complex data into transparent reports.", table_text_style)]
    ]

    table_width = letter[0] - 2 * inch
    col_widths = [table_width * 0.05,
                  table_width * 0.20,
                  table_width * 0.35,
                  table_width * 0.40]
    table = Table(data, colWidths=col_widths)
    table.setStyle(TableStyle([
        # All Borders
        ('GRID', (0,0), (-1,-1), 0.5, colors.grey),
        ('BOX', (0,0), (-1,-1), 1, colors.black),
        ('BACKGROUND', (0,0), (-1,0), ACCENT_BLUE),
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
        ('ALIGN', (0,0), (-1,0), 'CENTER'),
        ('VALIGN', (0,0), (-1,0), 'MIDDLE'),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0,0), (-1,0), 8),
        ('VALIGN', (0,1), (-1,-1), 'TOP'), # Top align all text for long descriptions
        ('ALIGN', (0,1), (0,-1), 'CENTER'), # Center align the '#' column
        ('LEFTPADDING', (0,0), (-1,-1), 4),
        ('RIGHTPADDING', (0,0), (-1,-1), 4),
        ('BACKGROUND', (0,1), (-1,-1), colors.white),
    ]))

    return [table, Spacer(1, 0.1*inch)]

def build_sharkrisk_report(filename="SharkRisk_Report.pdf", content_flowables=[]):
    doc = SimpleDocTemplate(
        filename, pagesize=letter,
        rightMargin=inch,
        leftMargin=inch,
        topMargin=0.5*inch,
        bottomMargin=0.5*inch,
    )
    story = []
    doc.build(story, onFirstPage=_header_footer, onLaterPages=_header_footer)
    page_width = letter[0] - 2 * inch

    if os.path.exists(LOGO_FILENAME):
        try:
            logo_reader = ImageReader(LOGO_FILENAME)
            logo_width, logo_height = logo_reader.getSize()
            aspect_ratio = logo_height / logo_width
            logo = Image(LOGO_FILENAME)
            logo.drawWidth = page_width * 0.5
            logo.drawHeight = logo.drawWidth * aspect_ratio
            logo.hAlign = 'CENTER'
            story.append(logo)
            story.append(Spacer(1,0.2*inch))

        except Exception:
            story.append(Paragraph("<b>[SharkRisk Logo Placeholder]</b>", styles['logo']))
    else:
        story.append(Paragraph("<b>[SharkRisk Logo Placeholder]</b>", styles['logo']))
        story.append(Spacer(1, 0.001 * inch))

    story.append(Spacer(1, 0.001 * inch))
    story.append(Paragraph("The first 100% Risk Management Python Library", styles['Tagline']))
    story.append(Spacer(1, 0.03*inch))
    story.append(Paragraph("<i>Risk is not a negative. It's an asset. Manage it that way.</i>", styles['Tagline']))

    story.append(Spacer(1, 0.5*inch))

    if os.path.exists(DASHBOARD_FILENAME):
        try:
            dashboard_reader = ImageReader(DASHBOARD_FILENAME)
            dash_width, dash_height = dashboard_reader.getSize()
            dash_aspect_ratio = dash_height / dash_width
            dashboard = Image(DASHBOARD_FILENAME)
            dashboard.drawWidth = page_width * 1.20
            dashboard.drawHeight = dashboard.drawWidth * dash_aspect_ratio
            dashboard.hAlign = 'CENTER'
            story.append(dashboard)
            story.append(Spacer(1, 0.3*inch))
        except Exception:
            story.append(Paragraph("<b>[RiskReporter Dashboard Image Placeholder]</b>", styles['Tagline']))
            story.append(Spacer(1, 0.5 * inch))
    else:
        story.append(Paragraph("<b>[RiskReporter Dashboard Image Placeholder]</b>", styles['Tagline']))
        story.append(Spacer(1, 0.1 * inch))

    story.append(Paragraph(f"<b>Date:</b> {datetime.now().strftime('%Y-%m-%d')}", styles['DateFooter']))

    story.append(PageBreak())

    story.extend(content_flowables)

    doc.build(story, onLaterPages=_header_footer)

    print(f"\nReport '{filename}' created successfully in the Colab file system.")
    try:
        files.download(filename)
        print("Download initiated.")
    except Exception:
        print("Error initiating download. File may still be available in Colab file browser.")

if __name__ == '__main__':

    execsum_content = [
        Paragraph("Executive Summary", styles['SectionHeading']),
        Paragraph("<b>SharkRisk</b> is a full-stack Python portfolio risk analysis solution engineered to transform risk management from a cost center into a strategic value driver. While the manual calculation and integration of complex risk measures (Greeks, VaR, GARCH) remain a significant bottleneck, SharkRisk automates the entire risk management lifecycle, delivering comprehensive risk metrics, AI-driven sentiment analysis, proactive hedging strategies, and compliance-ready automated reports.", styles['ExecutiveSummary']),
    ]

    execsum_content.extend(get_feature_impact_table())

    market_analysis_content = [
        Paragraph("Context: A Fragmented Market", styles['SectionHeading']),
        Paragraph("The modern investment landscape heavily relies on Python for its flexibility and robust quantitative capabilities. However, a significant gap exists between the tooling available for generic Asset Management and specialized Risk Management.", styles['BodyText']),
        Paragraph("The Python ecosystem is saturated with high-quality libraries focused on portfolio construction, backtesting, and basic financial modelling (Backtrader, Zipline, PyPortfolioOpt, …). While these tools are excellent for constructing and optimizing portfolios, they often treat risk management as a secondary function, providing only high-level metrics like standard deviation or Sharpe Ratios.", styles['BodyText']),
        Paragraph("In contrast to asset management, the specialized Risk Management space is highly fragmented. While several libraries offer pieces of the puzzle, none provide the complete, end-to-end suite required by institutional risk teams.", styles['BodyText']),
        Paragraph("<b>QuantLib/PyQL:</b> An industry-standard C++ library with Python bindings. While powerful, its complexity and steep learning curve often limit its use.", styles['BulletPoint']),
        Paragraph("<b>Riskfolio-Lib:</b> Primarily focused on portfolio optimization under various risk measures (like CVaR, etc.) but is not designed as a comprehensive risk reporting and hedging platform.", styles['BulletPoint']),
        Paragraph("<b>PyRisk:</b> Represents smaller, often specialized, or non-maintained efforts in specific areas of risk modeling.", styles['BulletPoint']),
        Paragraph("This fragmentation creates a major operational burden for risk managers and quantitative analysts. To produce a full risk report, a typical risk team is forced to switch between multiple disparate libraries:",styles['BodyText']),
        Paragraph("1.	Use <b>pandas</b> to import and clean the data.",styles['BodyText']),
        Paragraph("2.	Use <b>QuantLib</b> to calculate complex Greeks for derivatives exposure.", styles['BodyText']),
        Paragraph("3.	Use <b>Riskfolio-Lib</b> to optimize portfolio-level CVaR.", styles['BodyText']),
        Paragraph("4.	Use <b>proprietary code</b> to calculate Historical VaR and credit exposure.", styles['BodyText']),
        Paragraph("5.	Use <b>Matplotlib</b> to generate visualizations.", styles['BodyText']),
        Paragraph("This reliance on multiple tools leads to Integration Overheads writing glue code and ensuring data structures are compatible, Inconsistency Risk and Inefficiency that prevent real-time risk monitoring and proactive decision-making.", styles['BodyText']),
        Paragraph("SharkRisk directly addresses this critical need by offering a single, unified, and exhaustive library that integrates all necessary risk functions - from data import and Greek computation to AI sentiment analysis and automated regulatory reporting - into one coherent Python API. This integration eliminates the need to juggle and positions SharkRisk as the definitive full-stack risk management tool.", styles['BodyText']),
    ]

    main_functions_content = [
        Paragraph("Core Engine: 5 Main Functions", styles['SectionHeading']),
        Paragraph("The SharkRisk library is built around five integrated modules, each designed to automate and enhance a critical stage of the institutional risk management process.", styles['BodyText']),
    ]

    table_main_functions_content = [
    ]
    table_main_functions_content.extend(get_main_functions_table())

    main_functions_content_5 = [
        Paragraph("1. import.sharkrisk.py", styles['SubSectionHeading']),
        Paragraph("<b>Objective:</b> To establish a completely clean, validated, and statistically robust dataset, serving as the trusted foundation for all subsequent risk analysis.", styles['BodyText']),
        Paragraph("<b>Mechanism:</b> The system executes a series of automated checks: File & Format Checks (e.g., mandatory columns), Structural Validation (e.g., weights summing to 100%), Market Data Sourcing (fetching historical time series), and Initial Computations (e.g., Daily Returns and Correlation Matrix).", styles['BodyText']),
        Paragraph("<b>Output:</b> A validated, analysis-ready portfolio structure that guarantees mathematical soundness and data sufficiency for statistically robust risk metrics.", styles['BodyText']),

        Paragraph("2. metrics.sharkrisk.py", styles['SubSectionHeading']),
        Paragraph("<b>Objective:</b> To provide a comprehensive, institutional-grade assessment of portfolio and derivative vulnerability across market and non-market risks..", styles['BodyText']),
        Paragraph("<b>Mechanism:</b> The engine calculates exhaustive metric groups: Loss Quantification (VaR, CVaR, Max Drawdown), Derivative Sensitivity (Greeks: Delta, Gamma, Vega, Theta; and Systemic Risk (Credit Exposure, Correlation). These are augmented by visualizations like Volatility Contribution charts.", styles['BodyText']),
        Paragraph("<b>Output:</b> Critical loss estimates, essential derivative risk insights, and actionable visual analyses that meet regulatory capital requirements and fund mandates.", styles['BodyText']),

        Paragraph("3. sentiment.sharkrisk.py", styles['SubSectionHeading']),
        Paragraph("<b>Objective:</b> To provide an early warning system by integrating forward-looking, qualitative market factors into the quantitative risk assessment.", styles['BodyText']),
        Paragraph("<b>Mechanism:</b> The module acts as an AI Engine using Natural Language Processing (NLP) to scan public Data Sources (e.g., Reddit, Twitter) for portfolio assets. It classifies the discussion context to generate a quantitative Sentiment Score.", styles['BodyText']),
        Paragraph("<b>Output:</b> A quantitative sentiment score that serves as a unique risk factor input, offering a leading indicator to anticipate sentiment-driven volatility spikes often missed by purely historical models.", styles['BodyText']),

        Paragraph("4. hedging.sharkrisk.py", styles['SubSectionHeading']),
        Paragraph("<b>Objective:</b> It performs Risk Driver Identification based on contributions from VaR and Greeks. The system then generates Strategy Proposals for specific derivatives (e.g., put options or futures) tailored for Optimization Focus—neutralizing target risk factors while minimizing transaction costs.", styles['BodyText']),
        Paragraph("<b>Output:</b> Targeted, data-driven solutions for risk reduction that eliminate hedging guesswork and proactively protect capital against adverse market movements.", styles['BodyText']),

        Paragraph("5. report.sharkrisk.py", styles['SubSectionHeading']),
        Paragraph("<b>Objective:</b> To automate the final, time-consuming step of the risk management cycle, ensuring transparent communication and regulatory adherence.", styles['BodyText']),
        Paragraph("<b>Mechanism:</b> It creates a Comprehensive Output that automatically gathers charts, KPIs, risk metrics, and hedging recommendations. Reports are generated in a Compliance Ready structure and are easily Customizable for various internal and external stakeholder needs.", styles['BodyText']),
        Paragraph("<b>Output:</b> Professional, clear, and auditable documentation that streamlines communication for investors and regulators, saving countless hours in manual report assembly.", styles['BodyText']),

    ]

    main_functions_content_5.append(PageBreak())

    business_model_content = [
        Paragraph("Business Model", styles['SectionHeading']),
        Paragraph("SharkRisk utilizes a tiered subscription model to target institutional clients, ensuring scalable, reliable revenue. The Pro tier provides a clear path to scaling revenue as institutional usage is standardized under a predictable monthly fee. The model's value proposition is the replacement of fragmented, high-overhead workflow solutions with a single, unified API.", styles['BodyText']),
        Paragraph("The financial viability is established by relatively low fixed costs necessary for a software library. Upfront Development Costs primarily covered the initial integration of specialized data feeds, cloud development, and the labor required to build the core VaR/Greeks and NLP engines. Ongoing Monthly Operating Costs are dominated by external institutional data subscriptions and cloud hosting expenses for continuous risk computations.", styles['BodyText']),
        Paragraph("The model achieves high margins quickly because the marginal cost of serving an additional client is near zero. This high leverage is validated by a clear target: securing just 50 Pro clients allows the platform to reach a net monthly profit exceeding $20,000, demonstrating a strong financial path.", styles['BodyText']),
    ]

    next_steps_content = [
        Paragraph("Future Improvements", styles['SectionHeading']),
        Paragraph("Future development is focused on two areas. First, enhancing <b>Derivatives Data Importation</b> to support complex derivative fields (Strike, Expiry, Type) for accurate non-linear risk modeling. Second, implementing <b>Seamless Integration with the Bloomberg Terminal App (BLPAPI)</b> to access institutional-grade, real-time data, which is crucial for enabling true intra-day risk monitoring.", styles['BodyText']),
    ]

    references_content = [
        Paragraph("References", styles['SectionHeading']),
        Paragraph("<i>Click the reference to get access</i>", styles['BodyText']),
        Paragraph('<a href="https://theaiquant.medium.com/mastering-the-black-scholes-model-with-python-a-comprehensive-guide-to-option-pricing-11af712697b7" target="_blank">Mastering the Black-Scholes Model with Python: A Comprehensive Guide to Option Pricing</a>', styles['BulletPoint']),
        Paragraph('<a href="https://www.codearmo.com/python-tutorial/options-trading-greeks-black-scholes" target="_blank">Option Greeks by Analytic & Numerical Methods with Python</a>', styles['BulletPoint']),
        Paragraph('<a href="https://github.com/mattsta/pygreeks" target="_blank">pygreeks</a>', styles['BulletPoint']),
        Paragraph('<a href="https://www.pyquantnews.com/the-pyquant-newsletter/price-options-using-black-scholes-in-python" target="_blank">Price options using Black-Scholes in Python</a>', styles['BulletPoint']),
        Paragraph('<a href="https://sites.google.com/view/vinegarhill-financelabs/home?authuser=0" target="_blank">VinegarHill-FinanceLabs</a>', styles['BulletPoint']),
        Paragraph('<a href="https://kidquant.com/project/option-payoffs-black-scholes-and-the-greeks/" target="_blank">Option Payoffs, Black-Scholes, and the Greeks, KidQuant</a>', styles['BulletPoint']),
        Paragraph('<a href="https://www.quantlibguide.com/" target="_blank">QuantLib</a>', styles['BulletPoint']),
        Paragraph('<a href="https://github.com/lprtk/pyRisk" target="_blank">pyRisk</a>', styles['BulletPoint']),
        Paragraph('<a href="https://riskfolio-lib.readthedocs.io" target="_blank">Riskfolio-lib</a>', styles['BulletPoint']),
        Paragraph('<a href="https://pyportfolioopt.readthedocs.io" target="_blank">PyPortfolioOpt</a>', styles['BulletPoint']),
    ]

    code_content = [
        Paragraph("Access to the code", styles['SectionHeading']),
        Paragraph("The code represents an experimentation in implementing financial derivatives and interfaces in Python. Its development utilized provided course materials, external references, and generative AI tools.", styles['BodyText']),
        Paragraph("Please note the dashboard overview illustrates the forecasted final version of the tool.", styles['BodyText']),
        Paragraph('Click to access the <a href="https://github.com/jacquesescp/DSforAM_group10/blob/main/SharkRisk_Notebook.ipynb"><font color="red">first draft of the SharkRisk code</font></a>, <a href="https://github.com/jacquesescp/DSforAM_group10/blob/main/ReportGeneration.ipynb">the <font color="blue">report generation jupyter notebook code</font></a>. Each file can be found in the <a href="https://github.com/jacquesescp/DSforAM_group10/tree/main"><font color="green">repository of the project</font></a>', styles['BodyText']),
    ]

    full_report_content = (
        execsum_content +
        market_analysis_content +
        main_functions_content +
        table_main_functions_content +
        main_functions_content_5 +
        business_model_content +
        next_steps_content +
        references_content +
        code_content
    )

    build_sharkrisk_report(content_flowables=full_report_content)


Report 'SharkRisk_Report.pdf' created successfully in the Colab file system.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated.
